# Sentence Retrieval

In [44]:
import json,math,nltk

train_file = open("train.json",'r')
dev_file = open("dev.json",'r')
test_file=open("test.json",'r')
train = json.loads(train_file.read())
dev = json.loads(dev_file.read())
test = json.loads(test_file.read())
# train[0]['sentences']
# train[0]['qa']

In [45]:
from nltk.corpus import stopwords
# may be stem?

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()
stopword =  stopwords.words()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def doc_word_dict(doc):
    word_dict = set()
    for sent in doc['sentences']:
        for word in  word_tokenizer.tokenize(sent):
            word = lemmatize(word.lower())
            if word not in stopword:
                word_dict.add(word)
    return word_dict

In [46]:
def get_BOW(sent):
    term_dict={}
    for word in word_tokenizer.tokenize(sent):
        word = lemmatize(word.lower())
        if word not in stopword:
            term_dict[word]=term_dict.setdefault(word,0)+1
    return term_dict

def cal_BOW(doc):
    doc_term_matrix = [] 
    for sent in doc['sentences']:
        temp = get_BOW(sent)
        doc_term_matrix.append(temp)
    return doc_term_matrix

In [47]:
# def get_best_doc_num(query):
#     query =  transformer.transform(vectorizer.transform(get_BOW(query)))
#     f = lambda x: cos_distance(query.toarray(),term_matrix[x].toarray())
#     best_doc = min(range(term_matrix.shape[0]),key=f)
#     return best_doc

In [48]:
def get_best_doc_num2(query):
    query =  transformer.transform(vectorizer.transform(get_BOW(query)))
    result={}
    for x in range(term_matrix.shape[0]):
         result[x]=cos_distance(query.toarray(),term_matrix[x].toarray())
            
    minvalue=1
    first=0
    for item in result:
        if minvalue > result[item]:
            minvalue=result[item]
            first=item     
    del result[first]
    
    minvalue=1
    second=0
    for item in result:
        if minvalue > result[item]:
            minvalue=result[item]
            second=item     
    return first,second

In [49]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance

vectorizer = DictVectorizer()
transformer = TfidfTransformer(smooth_idf=False,norm=None)

match_sent= []
count = 0
for test_doc in test:
    print count
    count += 1
    doc_match_sent = []
    term_matrix = transformer.fit_transform(vectorizer.fit_transform(cal_BOW(test_doc)))
    for qa in test_doc['qa']:
        doc_match_sent.append(get_best_doc_num2(qa['question']))
    match_sent.append(doc_match_sent)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [50]:
len(match_sent)

42

# Entity Extraction

In [51]:
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('/Users/ZhangJiaWei/Downloads/stanford-ner-2016-10-31/classifiers/english.muc.7class.distsim.crf.ser.gz',
               '/Users/ZhangJiaWei/Downloads/stanford-ner-2016-10-31/stanford-ner.jar') 

test_tag = []
for i in range(len(match_sent)):
    test_sent_tag = []
    for j,k in match_sent[i]:
        test_sent_tag.append(word_tokenizer.tokenize(test[i]['sentences'][j] + ' ' + test[i]['sentences'][k]))
    test_sent_tag = st.tag_sents(test_sent_tag)
    test_tag.append(test_sent_tag)

In [52]:
len(test_tag[0])

287

In [53]:
def tune_other(tag_list):
    for i in range(len(tag_list)):
        for j in range(len(tag_list[i])):
            for k in range(len(tag_list[i][j])):
                term,tag = tag_list[i][j][k]
                if term!='' and (tag == "ORGANIZATION"  or (len(term)>0 and (term,tag)!=tag_list[i][j][0] and tag == 'O' and term[0].isupper())):
                    tag_list[i][j][k] = (term,"OTHER")

tune_other(test_tag)

In [54]:
def combine_entity(tag_list):
    for k in range(len(tag_list)):
        for i in range(len(tag_list[k])):
            j = 0
            while j < len(tag_list[k][i])-2:
                term,tag = tag_list[k][i][j]
                term_n,tag_n = tag_list[k][i][j+1]
                if tag == tag_n and tag != "O":
                    temp =  (term + " " + term_n,tag)
                    tag_list[k][i][j] = temp
                    del tag_list[k][i][j+1]
                j += 1

combine_entity(test_tag)

In [55]:
test_list = []
for k in test_tag:
    test_doc_list = []
    for i in k:
        test_ = []
        for term,tag in i:
            if term != '':
                test_.append(term)
        test_doc_list.append(test_)
    test_list.append(test_doc_list)

In [56]:
len(test_list[0])

287

In [57]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

tags = ["PERSON","LOCATION","NUMBER","OTHER"]
entity_pool = []
for k in test_tag:
    entity_doc_pool = []
    for i in k:
        sent_tag_dict = dict.fromkeys(tags,[])
        for j in range(len(i)):
            term,tag = i[j]
            if tag == "PERSON" or tag == "LOCATION" or tag == "OTHER":
                sent_tag_dict[tag] = sent_tag_dict[tag]+ [term]
            elif tag == "DATE" or tag == "TIME" or tag == "PERCENT" or hasNumbers(term):
                sent_tag_dict["NUMBER"] = sent_tag_dict["NUMBER"]+ [term]
        entity_doc_pool.append(sent_tag_dict)
    entity_pool.append(entity_doc_pool)

In [58]:
entity_pool[0]

[{'LOCATION': [], 'NUMBER': [u'350'], 'OTHER': [u'The'], 'PERSON': []},
 {'LOCATION': [], 'NUMBER': [u'350'], 'OTHER': [u'The'], 'PERSON': []},
 {'LOCATION': [u'Russia'],
  'NUMBER': [],
  'OTHER': [u'Christian',
   u'Holy Land',
   u'Ottoman Empire',
   u'Now',
   u'Sultan',
   u'Christian',
   u'Holy Land',
   u'Russian',
   u'Danubian'],
  'PERSON': []},
 {'LOCATION': [u'Russia', u'Palestine'],
  'NUMBER': [u'103'],
  'OTHER': [u'French',
   u'Catholics',
   u'Eastern Orthodox',
   u'Christians',
   u'Catholic',
   u'French',
   u'Roman Catholic'],
  'PERSON': [u'Marquis Charles', u'de La', u'Valette']},
 {'LOCATION': [u'Russia'],
  'NUMBER': [],
  'OTHER': [u'French',
   u'Catholics',
   u'Eastern Orthodox',
   u'Christians',
   u'By',
   u'Eastern Orthodox',
   u') Christian'],
  'PERSON': []},
 {'LOCATION': [u'Russia'],
  'NUMBER': [],
  'OTHER': [u'I', u'Christians', u'Orthodox', u'Christian'],
  'PERSON': [u'Nicholas', u'Napoleon III']},
 {'LOCATION': [],
  'NUMBER': [u'350'],


# Answer Ranking

In [59]:
import operator
import nltk

# A simple rule-based question type classifier based on key words 
def get_question_type(question):
    # TODO: HAND-CODED, NEED TO BE REFINED!!
    # TODO: need to low-case to compare?
    
    type_rules = {'PERSON':["Who", "Whose", "Whom"],
                  'LOCATION':["Where"],
                  'NUMBER':["When", "few", "little", "much", "many","size",
                            "young", "old", "long", "year", "years"]
                 }
    q_type = None
    for question_type, key_words in type_rules.items():
        for key_word in key_words:
            if key_word in question:
                q_type = question_type
                break
        if q_type == None:
            q_type = 'OTHER'
    return q_type

In [60]:
from nltk import word_tokenize

# among entities of the same type, the prefered entity should be 
# the one which is closer in the sentence to a open-class word
# from the question.
# ----> nouns, verbs, adjectives, and adverbs.
def get_preferred_entity(entity_list, sentence, question):
    preferred_entity = None
    sentence_text = sentence
    question_text = word_tokenize(question)
    sentence_tag = nltk.pos_tag(sentence_text,tagset='universal')
    question_tag = nltk.pos_tag(question_text,tagset='universal')
    
    # initialize a list for comparing, and set all elements as 0
    is_open_word = [0] * len(sentence_text)
    # find an open word in the question
    for word, tag in question_tag:
        if tag in ['ADJ','NOUN','VERB','ADV']:
            # if the open word appears in the sentence, then mark as 1
            for i in range(len(sentence_text)):
                if sentence_text[i] == word:
                    is_open_word[i] = 1
#     print is_open_word
    
    # find the closest distance to an open-class word for an entity
    def get_distance(entity):
        # get the position of entity, and find the open class word 
        # from the nearest at both sides
        distance = None
        position = sentence_text.index(entity)
        for i in range(1, len(sentence_text)):
            if position - i >= 0:
                if is_open_word[position - i] == 1:  # find an open-class word on the left
                    distance = i
                    break
                elif position + i < len(is_open_word):  # find an open-class word on the right
                    if is_open_word[position + i] == 1:
                        distance = i
                        break
                else:
                    distance = len(sentence_text) + 1  # didn't find open-class words
        return distance
    
    # get distance for each entity and choose the best one
    all_distance = []
    for entity in entity_list:
        all_distance.append(get_distance(entity))
        preferred_entity = entity_list[all_distance.index(min(all_distance))]

    return preferred_entity
    

In [61]:
def get_ranked_ans(entities_dic, question, sentence):
    
    # identify if the entity set is empty. If True, return nothing
    is_empty = True
    for values in entities_dic.values():
        if len(values) != 0:
            is_empty = False
            
    if is_empty == False:
        q_type = get_question_type(question)
        tmp_rank = {}
        for ent_type,entities in entities_dic.items():
            # answers whose content words all appear in the question should be ranked lowest.
            for entity in entities:
                if entity in question:
                    tmp_rank[entity] = tmp_rank.setdefault(entity,0) - 1
            # Answers which match the question type should be ranked higher than those that don't
            if ent_type == q_type:
                for entity in entities:
                    tmp_rank[entity] = tmp_rank.setdefault(entity,0) + 1
                ######## TODO: Apply this to all types?
                    # entity closer in the sentence to a closed-class word should be preferred
                    tmp_rank[get_preferred_entity(entities, sentence, question)] = tmp_rank.setdefault(entity,0) + 1
        # sort and choose the best answer
        sorted_ans = sorted(tmp_rank.items(), key=operator.itemgetter(1), reverse=True)
        #####TODO: bug here. list out of index??? why?
        if len(sorted_ans) != 0:
            best_ans = sorted_ans[0][0]
        else:
            best_ans = ''
        return best_ans
    else:
        return ''

In [64]:
count = 1
with open("result.txt",'w') as output_file:
    for i in range(len(match_sent)):
        for j in range(len(match_sent[i])):
    #         cor_answer = test[i]["qa"][j]['answer']
            result = get_ranked_ans(entity_pool[i][j], test[i]["qa"][j]['question'], test_list[i][j])
            q_id = test[i]["qa"][j]['id']
            count += 1
            string = str(q_id)+","+result.encode('utf-8')+"\n"
            output_file.write(string)
    #         if result == cor_answer:
    #             sum += 1
    # print sum

In [65]:
num = 0
for i in test:
    for j in i["qa"]:
        num += 1
# print (sum+0.0)/num

In [ ]:
# import gensim
# from gensim import corpora
# import math

# class BM25 :
#     def __init__(self, fn_docs, delimiter='|') :
#         self.dictionary = corpora.Dictionary()
#         self.DF = {}
#         self.delimiter = delimiter
#         self.DocTF = []
#         self.DocIDF = {}
#         self.N = 0
#         self.DocAvgLen = 0
#         self.fn_docs = fn_docs
#         self.DocLen = []
#         self.buildDictionary()
#         self.TFIDF_Generator()

#     def buildDictionary(self) :
#         raw_data = []
#         for line in file(self.fn_docs) :
#             raw_data.append(line.strip().split(self.delimiter))
#         self.dictionary.add_documents(raw_data)

#     def TFIDF_Generator(self, base=math.e) :
#         docTotalLen = 0
#         for line in file(self.fn_docs) :
#             doc = line.strip().split(self.delimiter)
#             docTotalLen += len(doc)
#             self.DocLen.append(len(doc))
#             #print self.dictionary.doc2bow(doc)
#             bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
#             for term, tf in bow.items() :
#                 if term not in self.DF :
#                     self.DF[term] = 0
#                 self.DF[term] += 1
#             self.DocTF.append(bow)
#             self.N = self.N + 1
#         for term in self.DF:
#             self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
#         self.DocAvgLen = docTotalLen / self.N

#     def BM25Score(self, Query=[], k1=1.5, b=0.75) :
#         query_bow = self.dictionary.doc2bow(Query)
#         scores = []
#         for idx, doc in enumerate(self.DocTF) :
#             commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
#             tmp_score = []
#             doc_terms_len = self.DocLen[idx]
#             for term in commonTerms :
#                 upper = (doc[term] * (k1+1))
#                 below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
#                 tmp_score.append(self.DocIDF[term] * upper / below)
#             scores.append(sum(tmp_score))
#         return scores

#     def TFIDF(self) :
#         tfidf = []
#         for doc in self.DocTF :
#             doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
#             doc_tfidf.sort()
#             tfidf.append(doc_tfidf)
#         return tfidf

#     def Items(self) :
#         # Return a list [(term_idx, term_desc),]
#         items = self.dictionary.items()
#         items.sort()
#         return items

# if __name__ == '__main__' :
#     fn_docs = 'mycorpus.txt'
#     bm25 = BM25(fn_docs, delimiter=' ')
#     Query = 'The intersection graph of paths in trees survey Graph'
#     Query = Query.split()
#     scores = bm25.BM25Score(Query)
#     print scores
#     tfidf = bm25.TFIDF()
# #     print bm25.Items()
#     for i, tfidfscore in enumerate(tfidf):
#         print i, tfidfscore
